In [1]:
import json
from tqdm.auto import tqdm 
from elasticsearch import Elasticsearch
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer

In [2]:
with open('../documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [3]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [8]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [9]:
es_client = Elasticsearch('http://localhost:9200')

In [10]:
es_client.info()

ObjectApiResponse({'name': 'c6a25e11fc01', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'q3x2jwyeRk-SkmkPtmNUgg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [11]:
index_name = "course-questions"

In [12]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [13]:
query = 'I just disovered the course. Can I still join it?'

In [14]:
index_name = "course-questions"

In [17]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"], #question is 3 times more important that text or question
                        "type": "best_fields"
                    }
                },
                # "filter": {
                #     "term": {
                #         "course": "data-engineering-zoomcamp"
                #     }
                # }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    print(response)
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [18]:
elastic_search(query)

{'took': 15, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 884, 'relation': 'eq'}, 'max_score': 79.96579, 'hits': [{'_index': 'course-questions', '_id': 'XKN1k5ABfPSlOFwRvWgg', '_score': 79.96579, '_ignored': ['text.keyword'], '_source': {'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'question': 'The course has already started. Can I still join it?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'naN1k5ABfPSlOFwRpGYU', '_score': 72.849266, '_source': {'text': "Yes, even if you don't register, you're still

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Welcome to the course! Go to the course page (http

In [19]:
with open('../huggingface.txt', 'r') as file:
    token = file.read().strip()

In [24]:
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-v0.1", device_map="auto", load_in_4bit=True, token=token
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [28]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1", padding_side="left", token=token)


In [29]:
model_inputs = tokenizer(["A list of colors: red, blue"], return_tensors="pt").to("cuda")

In [31]:
model_inputs

{'input_ids': tensor([[    1,   330,  1274,   302,  9304, 28747,  2760, 28725,  5045]],
       device='cuda:1'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:1')}

In [32]:
generated_ids = model.generate(**model_inputs)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/home/tania/anaconda3/lib/python3.11/site-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/tania/anaconda3/lib/python3.11/site-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


In [33]:
generated_ids

tensor([[    1,   330,  1274,   302,  9304, 28747,  2760, 28725,  5045, 28725,
          5344, 28725,  9684, 28725, 14545, 28725, 19435, 28725, 12937, 28725]],
       device='cuda:1')

In [36]:
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

'A list of colors: red, blue, green, yellow, orange, purple, pink,'

In [48]:
def build_prompt(query, search_results):
    prompt_template = """
        QUESTION: {question}

        CONTEXT: {context}

        ANSWER:
        """.strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    model_inputs = tokenizer([prompt], return_tensors="pt").to("cuda") 
    generated_ids = model.generate(**model_inputs, max_length=700, temperature=0.7, top_p=0.95, num_return_sequences=1)
    result = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return result

In [49]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [50]:
rag(query)

/home/tania/anaconda3/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/tania/anaconda3/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'took': 6, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 884, 'relation': 'eq'}, 'max_score': 79.96579, 'hits': [{'_index': 'course-questions', '_id': 'XKN1k5ABfPSlOFwRvWgg', '_score': 79.96579, '_ignored': ['text.keyword'], '_source': {'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'question': 'The course has already started. Can I still join it?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'naN1k5ABfPSlOFwRpGYU', '_score': 72.849266, '_source': {'text': "Yes, even if you don't register, you're still 

"QUESTION: I just disovered the course. Can I still join it?\n\n        CONTEXT: section: General course-related questions\nquestion: The course has already started. Can I still join it?\nanswer: Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.\n\nsection: General course-related questions\nquestion: Course - Can I still join the course after the start date?\nanswer: Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.\n\nsection: General course-related questions\nquestion: I just joined. What should I do next? H